In [320]:
import os
import re
import io
from math import log
from math import e
import sys
from collections import Counter
import nltk
from nltk.corpus import stopwords


In [321]:
#Read data set, extract each words in list for each file
import re
def readFile(path):
    vocab = []
    dictionary = {}
    for file in os.listdir(path):
        f = open(path+"/"+file,encoding = "ISO-8859-1")
        text = f.read()
        text = re.sub('[^a-zA-Z]', ' ', text)
        words_list = text.strip().split()
        dictionary[file] = words_list
        vocab.extend(words_list)
    return vocab, dictionary

In [322]:
#Calculate proir probability and conditional probability, store them into dictionary
def naiveBayestraining(countSpam, countHam, SpamFreq, HamFreq, SpamList, HamList, List):
    
    addOne = 1
    condiprobSpam = {}
    condiprobHam = {}

    for word in List:
        occurrence = 0
        if word in SpamFreq:
            occurrence = SpamFreq[word]
        condiprob1 = 1.0 * (occurrence + addOne) / (len(SpamList) + len(List))
        condiprobSpam[word] = condiprob1

    for word in List:
        occurrence = 0
        if word in HamFreq:
            occurrence = HamFreq[word]
        condiprob2 = 1.0 * (occurrence + addOne) / (len(HamList) + len(List))
        condiprobHam[word] = condiprob2
    spamPrior = 1.0 * countSpam / (countSpam + countHam)
    hamPrior = 1.0 * countHam / (countSpam + countHam)
    return spamPrior, hamPrior, condiprobSpam, condiprobHam

In [323]:
#Classify the type for test data set and calculate the accurancy
def naiveBayes(spamPrior, hamPrior, condiprobSpam, condiprobHam, spamDic, hamDic, List):
    dictionary = [spamDic, hamDic]
    total = len(spamDic) + len(hamDic)
    correct = 0
    for i in range(len(dictionary)):
        for fileName in dictionary[i]:
            score1 = log(spamPrior)
            score2 = log(hamPrior)
            for word in dictionary[i][fileName]:
                if word in List:
                    score1 += log(condiprobSpam[word])
                    score2 += log(condiprobHam[word])
            if score1 >= score2 and i == 0:
                correct +=1
            elif score1 <= score2 and i == 1:
                correct +=1
    return 100 * correct/total

In [324]:
if __name__ == "__main__":
    trainSpamPath = r'C:/Users/PranavD/Desktop/Semester2/Machine Learning/Assignment 2/train/spam'
    trainHamPath = r'C:/Users/PranavD/Desktop/Semester2/Machine Learning/Assignment 2/train/ham'
    testSpamPath = r'C:/Users/PranavD/Desktop/Semester2/Machine Learning/Assignment 2/test/spam'
    testHamPath = r'C:/Users/PranavD/Desktop/Semester2/Machine Learning/Assignment 2/test/ham'
    stopwords = set(stopwords.words('english'))
    willRemoveStopWords = 'no'

    if willRemoveStopWords == 'yes':
        trainspamList = []
        trainspamDic = {}
        for file in os.listdir(trainSpamPath):
            #print("entered 1 loop")
            f = open(trainSpamPath+"/"+file,encoding = "ISO-8859-1")  
            text = f.read()
            text = re.sub('[^a-zA-Z]', ' ', text)
            words_list = text.strip().split()
            words_new = []
        for word in words_list:
            if word not in stopwords:
                words_new.append(word)
        trainspamDic[file] = words_new
        trainspamList.extend(words_new)
        
        
        trainhamList = []
        trainhamDic = {}
        for file in os.listdir(trainHamPath):
            #print("Entered 2 loop")
            f = open(trainHamPath+"/"+file,encoding = "ISO-8859-1") 
            text = f.read()
            text = re.sub('[^a-zA-Z]', ' ', text)
            words_list = text.strip().split()
            words_new = []
        for word in words_list:
            if word not in stopwords:
                words_new.append(word)
        trainhamDic[file] = words_new
        trainhamList.extend(words_new)
           
        
    else:
        trainspamList, trainspamDic = readFile(trainSpamPath)
        trainhamList, trainhamDic = readFile(trainHamPath)

    testSpamList, testspamDic = readFile(testSpamPath)
    testHamList, testhamDic = readFile(testHamPath)

    numSpam = len(trainspamDic)
    numHam = len(trainhamDic)

    spamFreq = Counter(trainspamList)
    hamFreq = Counter(trainhamList)
    List = list(set(trainspamList)|set(trainhamList))
    spamPrior, hamPrior, condiprob1, condiprob2 = naiveBayestraining(numSpam, numHam,spamFreq, hamFreq, trainspamList, trainhamList,List)
    Accuracy =naiveBayes(spamPrior, hamPrior, condiprob1, condiprob2, testspamDic, testhamDic, List)
    print (Accuracy)

94.76987447698745
